In [14]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set(style='whitegrid')

from collections import defaultdict

from sklearn.model_selection import train_test_split, cross_val_score

from keras.optimizers import Adagrad
from tslearn.shapelets import ShapeletModel
from tslearn.shapelets import grabocka_params_to_shapelet_size_dict
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

from pyts.datasets import load_basic_motions

from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

In [15]:
# - Attributo 'Occupancy' dato dalla media oraria dei valori di occupancy

dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv('dataset.csv', parse_dates=True, index_col='date', date_parser=dateparse)
df.head()

C:\Users\dalla\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,day,hour,minute,day_minute,cumulative_hour,cumulative_minute,weekend
date,,,,,,,,,,,,,,
2015-02-02 14:19:00,0,23.7000,26.272,585.200000,749.200000,0.004764,1,2,14,19,859,0,0,0
2015-02-02 14:19:59,1,23.7180,26.290,578.400000,760.400000,0.004773,1,2,14,20,860,0,1,0
2015-02-02 14:21:00,2,23.7300,26.230,572.666667,769.666667,0.004765,1,2,14,21,861,0,2,0
2015-02-02 14:22:00,3,23.7225,26.125,493.750000,774.750000,0.004744,1,2,14,22,862,0,3,0
2015-02-02 14:23:00,4,23.7540,26.200,488.600000,779.000000,0.004767,1,2,14,23,863,0,4,0


In [16]:
l = []
CO2 = []
Temperature = []
Light = []
HumidityRatio = []
Humidity = []
for i, date in enumerate(list(df.index)):
    s = ''
    date = str(date)
    d = date.split(' ') #d e' la lista [data, orario]
    d1 = d[0].split('-') #d1 e' la lista [anno, mese, giorno]
    d2 = d[1].split(':') #d2 e' la lista [ore, minuti, secondi]
    anno = d1[0] #anno e' la stringa contenente l'anno
    mese = d1[1] #e' una stringa
    giorno = d1[2] #e' una stringa
    ora = d2[0] #e' una stringa
    minuto = d2[1] #e' una stringa
    secondo = d2[2] #e' una stringa
    if(giorno == '02' or giorno == '04' or giorno == '10' or giorno == '11' or giorno == '18'): # giorni non completi da saltare
        continue
    #il valore di secondo puo' essere '00' oppure '59'
    if(secondo != '00'): #devo incrementare i minuti
        secondo = '00'
        if(int(minuto) >= 9 and int(minuto) != 59):
            minuto = str(int(minuto) + 1)
        elif(int(minuto) >= 9 and int(minuto) == 59): 
            minuto = '00'
            if(int(ora) >= 9 and int(ora) != 23):
                ora = str(int(ora) + 1)
            elif(int(ora) >= 9 and int(ora) == 23):
                ora = '00'
                if(int(giorno) >= 9):
                    giorno = str(int(giorno) + 1)
                else:
                    giorno = '0' + str(int(giorno) + 1)
            else: #(int(ora) < 9):
                ora = '0' + str(int(ora) + 1)
        elif(int(minuto) < 9):
            minuto = '0' + str(int(minuto) + 1)
    s = anno + '-' + mese + '-' + giorno + ' ' + ora + ':' + minuto + ':' + secondo #s e' la stringa relativa alla data
                                                                                    #completa che inserisco nella lista l
    Light.append(df['Light'].iloc[i])
    HumidityRatio.append(df['HumidityRatio'].iloc[i])
    Humidity.append(df['Humidity'].iloc[i])
    Temperature.append(df['Temperature'].iloc[i])
    CO2.append(df['CO2'].iloc[i]) #mi salvo i valori di CO2
    l.append(s) #lista completa di tutte le date correttamente formattate

In [17]:
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  #Normalizziamo TUTTO il dataset
TUTTO = []
TUTTO.append(CO2)
TUTTO.append(Light)
TUTTO.append(HumidityRatio)
TUTTO.append(Humidity)
TUTTO.append(Temperature)

In [18]:
TUTTO = scaler.fit_transform(np.array(TUTTO))

In [19]:
TUTTO[0][0]

array([-0.76936498])

In [20]:
# crea un dataset di 6 ore
def build_dataset(df, attribute, class_name):
    X = [] #Lista lunga 5 di liste lunghe 360
    y = []
    for d in [3, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17]:
        day = df.loc[df['day'] == d]
        for h in range(0, 4):
            if h != 0:
                hour = day.loc[(day['hour'] >= h * 6) & (day['hour'] < (h+1) * 6)]
            else:
                hour = day.loc[day['hour'] <= 5]
            X.append(hour[attribute].values)
            y.append(1 if hour[class_name].mean() >= 0.50 else 0)
    X = np.array(X)
    y = np.array(y)
    return X, y

In [21]:
X = []
for i in range(5):
    l = []
    for j in range(17280):
        l.append(float(TUTTO[i][j]))
    X.append(l)
X = np.array(X)
X.shape

(5, 17280)

In [22]:
X_train = []
X_test = []

inizio = 0
fine = 360
for i in range(48):
    tabella = []
    for j in range(5):
        tmp = []
        for k in range(inizio, fine):
            if(fine > 17280):
                break
            tmp.append(X[j][k])
        tabella.append(tmp)
    inizio = fine 
    fine += 360

    if(i > 32):
        X_test.append(tabella)
    else:
        X_train.append(tabella)

X_train = np.array(X_train)
X_test = np.array(X_test)

# y dal notebook TS4: [0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0]

y = [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
y_train = y[:33]
y_test =  y[33:]

print(X_train.shape)
print(X_test.shape)
print(len(y_train))
print(len(y_test))

(33, 5, 360)
(15, 5, 360)
33
15


In [23]:
X_train = X_train.reshape(33, 360, 5)
X_test = X_test.reshape(15, 360, 5)

In [24]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_train)), X_train.shape[2] 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)
print("N. FEATURES: ", n_features)

TIMESTEPS:  360
N. LABELS:  2
N. FEATURES:  5


In [28]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

In [29]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Conv1D, Conv2D, MaxPool2D, Flatten, Dropout, LeakyReLU, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [58]:
from itertools import product
l= [1, 2, 3, 4, 5]
len(list(product(l, repeat = 2)))

25

In [68]:
%%time
summary = []
conta_iterazione = 0
from itertools import product

learn_rate = [0.01, 0.001, 0.0001]
act_f = ['relu', 'tanh', 'linear', 'softsign', 'softplus']
for act in product(act_f, repeat = 2):
    for l in learn_rate:
        model = Sequential()
        
        model.add(Conv1D(filters=16, kernel_size=8, activation=act[0], input_shape=(360, 5)))
        model.add(BatchNormalization())
        model.add(Activation(act[0]))
        
        model.add(Dropout(0.3))
        
        model.add(Conv1D(filters=32, kernel_size=5, activation=act[1]))
        model.add(BatchNormalization())
        model.add(Activation(act[1]))
        
        model.add(Dropout(0.3))
        
        model.add(GlobalAveragePooling1D())
        
        model.add(Dense(2, activation='sigmoid'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        
        
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=l)
        mc = ModelCheckpoint('best_model_cnn.h5', monitor='val_loss', save_best_only=True)
        
        callbacks = [rlr, mc]
        
        batch_size = 9
        mini_batch_size = int(min(X_train.shape[0]/10, batch_size))
        
        history_cnn = cnn.fit(X_train, y_train, epochs=100, batch_size=mini_batch_size, callbacks=callbacks,
                              validation_data=(X_val_cnn, y_val_cnn), verbose = 0).history
        
        loss_curve = history_cnn['loss'] # per la loss curve
        
        y_pred = np.argmax(cnn.predict(X_test), axis=1)
        
        print('\n\n============= MODEL {} ==================='.format(conta_iterazione))
        print("activation functions: {}, {}, learning rate: {}".format(act[0], act[1], l))
        print('Accuracy Model {} {}'.format(conta_iterazione, accuracy_score(y_test, y_pred)))
        print('F1-score Model {} {}'.format(conta_iterazione, f1_score(y_test, y_pred, average=None)))
        print('Evaluation on test set', cnn.evaluate(X_test, y_test))
        print('Real values:     ', y_test)
        print('Predicted values:', list(y_pred))
        
        conta_iterazione += 1
        report = classification_report(y_test, y_pred, output_dict=True)
        
        summary.append({'iterazione' : conta_iterazione, 
                    'activation functions' : act,
                    'learning rate' : l,
                    'test accuracy' : accuracy_score(y_test, y_pred),
                    'f1-score [0]' : f1_score(y_test, y_pred, average=None)[0],
                    'f1-score [1]' : f1_score(y_test, y_pred, average=None)[1],
                    'precision [1]' : report['1']['precision'],
                    'recall [1]' : report['1']['recall'],
                    'model' : model,
                    'loss curve' : loss_curve})



============= MODEL 0 ===================
activation functions: relu, relu, learning rate: 0.01
Accuracy Model 0 1.0
F1-score Model 0 [1. 1.]
15/15 [==============================] - 0s 333us/step
Evaluation on test set [0.11962096393108368, 1.0]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]


============= MODEL 1 ===================
activation functions: relu, relu, learning rate: 0.001
Accuracy Model 1 0.8666666666666667
F1-score Model 1 [0.9 0.8]
15/15 [==============================] - 0s 333us/step
Evaluation on test set [0.25566211342811584, 0.8666666746139526]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]


============= MODEL 2 ===================
activation functions: relu, relu, learning rate: 0.0001
Accuracy Model 2 0.8
F1-score Model 2 [0.84210526 0.72727273]
15/15 [==============================] - 0s 467us/s



============= MODEL 21 ===================
activation functions: tanh, linear, learning rate: 0.01
Accuracy Model 21 0.9333333333333333
F1-score Model 21 [0.95652174 0.85714286]
15/15 [==============================] - 0s 267us/step
Evaluation on test set [0.14860913157463074, 0.9333333373069763]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]


============= MODEL 22 ===================
activation functions: tanh, linear, learning rate: 0.001
Accuracy Model 22 0.8666666666666667
F1-score Model 22 [0.9 0.8]
15/15 [==============================] - 0s 267us/step
Evaluation on test set [0.3448139727115631, 0.8666666746139526]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]


============= MODEL 23 ===================
activation functions: tanh, linear, learning rate: 0.0001
Accuracy Model 23 0.9333333333333333
F1-score Model 23 

Evaluation on test set [0.6549387574195862, 0.800000011920929]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1]


============= MODEL 42 ===================
activation functions: linear, softplus, learning rate: 0.01
Accuracy Model 42 0.8666666666666667
F1-score Model 42 [0.91666667 0.66666667]
15/15 [==============================] - 0s 333us/step
Evaluation on test set [0.16668947041034698, 0.8666666746139526]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]


============= MODEL 43 ===================
activation functions: linear, softplus, learning rate: 0.001
Accuracy Model 43 1.0
F1-score Model 43 [1. 1.]
15/15 [==============================] - 0s 333us/step
Evaluation on test set [0.06855877488851547, 1.0]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,



============= MODEL 62 ===================
activation functions: softplus, relu, learning rate: 0.0001
Accuracy Model 62 1.0
F1-score Model 62 [1. 1.]
15/15 [==============================] - 0s 400us/step
Evaluation on test set [0.07181552052497864, 1.0]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]


============= MODEL 63 ===================
activation functions: softplus, tanh, learning rate: 0.01
Accuracy Model 63 1.0
F1-score Model 63 [1. 1.]
15/15 [==============================] - 0s 333us/step
Evaluation on test set [0.105434849858284, 1.0]
Real values:      [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Predicted values: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]


============= MODEL 64 ===================
activation functions: softplus, tanh, learning rate: 0.001
Accuracy Model 64 0.9333333333333333
F1-score Model 64 [0.95238095 0.88888889]
15/15 [==============================] - 0s 400us

In [66]:
bel_modello = summary[0]

In [67]:
bel_modello

{'iterazione': 0,
 'activation functions': ('relu', 'relu'),
 'learning rate': 0.01,
 'test accuracy': 0.9333333333333333,
 'f1-score [0]': 0.9565217391304348,
 'f1-score [1]': 0.8571428571428571,
 'precision [1]': 1.0,
 'recall [1]': 0.75,
 'model': <keras.engine.sequential.Sequential at 0x277ce479988>,
 'loss curve': [0.2940310841569534,
  0.3101297854804076,
  0.2886527937192183,
  0.2519878074526787,
  0.24680009255042443,
  0.2717310284765867,
  0.2157920982975226,
  0.2783289889876659,
  0.28329251821224505,
  0.2920550749852107,
  0.27444952296522945,
  0.15667369703833872,
  0.28601308740102327,
  0.25716455166156477,
  0.38740251987026286,
  0.3120351823476645,
  0.226505030806248,
  0.22155331132503656,
  0.22852109687832686,
  0.42899350076913834,
  0.42377256659361034,
  0.2815492611664992,
  0.19509403121012908,
  0.3518416185218554,
  0.19979271235374305,
  0.2135667193394441,
  0.30075334356381345,
  0.20701422485021445,
  0.22593763957803065,
  0.1466763331913031,
  0.1

In [76]:
for iteration in summary:
    if iteration['test accuracy'] == 1:
        print("Iterazione: ", iteration['iterazione'])
        print("Activation fuctions: ", iteration['activation functions'])
        print("Learning rate: ", iteration['learning rate'], end = '\n\n')

Iterazione:  1
Activation fuctions:  ('relu', 'relu')
Learning rate:  0.01

Iterazione:  5
Activation fuctions:  ('relu', 'tanh')
Learning rate:  0.001

Iterazione:  7
Activation fuctions:  ('relu', 'linear')
Learning rate:  0.01

Iterazione:  8
Activation fuctions:  ('relu', 'linear')
Learning rate:  0.001

Iterazione:  10
Activation fuctions:  ('relu', 'softsign')
Learning rate:  0.01

Iterazione:  11
Activation fuctions:  ('relu', 'softsign')
Learning rate:  0.001

Iterazione:  12
Activation fuctions:  ('relu', 'softsign')
Learning rate:  0.0001

Iterazione:  13
Activation fuctions:  ('relu', 'softplus')
Learning rate:  0.01

Iterazione:  14
Activation fuctions:  ('relu', 'softplus')
Learning rate:  0.001

Iterazione:  16
Activation fuctions:  ('tanh', 'relu')
Learning rate:  0.01

Iterazione:  17
Activation fuctions:  ('tanh', 'relu')
Learning rate:  0.001

Iterazione:  18
Activation fuctions:  ('tanh', 'relu')
Learning rate:  0.0001

Iterazione:  19
Activation fuctions:  ('tanh', 